+ https://github.com/JuliaAcademy/DataScience/blob/master/08.%20Graphs.ipynb

In [ ]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()
Pkg.add("MatrixNetworks")
Pkg.resolve()

In [ ]:
using LightGraphs
using MatrixNetworks
using VegaDatasets
using DataFrames
using SparseArrays
using LinearAlgebra
using Plots
using VegaLite

In [ ]:
airports = dataset("airports")# dati su aereoporti
flightsairport = dataset("flights-airport");#connessioni tra aereoporti

In [ ]:
# sono var di tipo vegadataset, non dataframe, convertiamo a dataframe
flightsairportdf = DataFrame(flightsairport)
airportsdf = DataFrame(airports);

In [ ]:
allairports = vcat(flightsairportdf[!,:origin],flightsairportdf[!,:destination])#concatena colonne origine e destinazione

uairports = unique(allairports);
# ottiene insieme di tutti gli aereoporti che sono origine o destinazione di un collegamento tra aereoporti

In [ ]:
subsetairports = map(i->findfirst(airportsdf[!, :iata].==uairports[i])#trova datidel df aereoporti dell'aereoporto i-esimo dell'array 
    #degli aereoporti unici per cui esiste collegamento
    ,1:length(uairports))#scorre ogni aereoporto per cui esiste collegamento
airportsdf_subset = airportsdf[subsetairports,:];#converte in df

In [ ]:
#creiamo una mat di adiacenza(adiacenza(a,b)= se esiste collegamento tra aereop a e aereop b)

ei_ids = findfirst.(isequal.(flightsairportdf[!,:origin]), [uairports])
ej_ids = findfirst.(isequal.(flightsairportdf[!,:destination]), [uairports])
edgeweights = flightsairportdf[!,:count]
;
#abbiamo un numero molto alto di aereoporti rispetto ai collegamenti, quindi usiamo matrice sparsa per salvare mat adiac
A = sparse(ei_ids,ej_ids,1,length(uairports),length(uairports))
A = max.(A,A')# NB: creiamo grafo non direzionale, adiacenza(a,b) == adiacenza(b,a)

In [ ]:
spy(A)#funzione su matrici sparse

In [ ]:
# yup, sparsa

cc = scomponents(A) # ci da i componenti connessi del grafo
@show cc.sizes #mostra le grandezze delle componenti connesse, 
# gli eaereoporti formano una sola componente connessa quindi sizes darà  un int = length(unique airports)
# length(cc.sizes) = numero di componenti connesse

degrees = sum(A,dims=2)[:]#array dei gradi degli aereoporti
p1 = plot(sort(degrees,rev=true),ylabel="log degree",legend=false,yaxis=:log)
p2 = plot(sort(degrees,rev=true),ylabel="degree",legend=false)
plot(p1,p2,size=(600,300))

In [ ]:
# la distribuzione della rete di aereoporti sembra seguire distribuzione esponenziale
# ciò può essere utile per costruire algoritmi sul grafo più performanti
maxdegreeid = argmax(degrees)
uairports[maxdegreeid]

In [ ]:
# l'aereoportov di atlanta è l'aereoporto connesso al maggior nmuero di aereoporti
us10m = dataset("us-10m")#rappresentazione dell'america come df
@vlplot(width=500, height=300) +
@vlplot(#disegna cartina degli stati uniti
    mark={
        :geoshape,
        fill=:lightgray,
        stroke=:white
    },
    data={
        values=us10m,
        format={
            type=:topojson,
            feature=:states
        }
    },
    projection={type=:albersUsa},
) +
@vlplot(#scatterplot degli aereoporti (nodi)
    :circle,
    data=airportsdf_subset,
    projection={type=:albersUsa},
    longitude="longitude:q",
    latitude="latitude:q",
    size={value=10},
    color={value=:steelblue}
)+
@vlplot(#crea le linee del grafo
    :rule,
    data=flightsairport,
    transform=[
        {filter={field=:origin,equal=:ATL}},#mostra solo collegamenti da atlanta
        {#prende origine e scompone in latitudine e longitudine
            lookup=:origin,
            from={
                data=airportsdf_subset,
                key=:iata,
                fields=["latitude", "longitude"]
            },
            as=["origin_latitude", "origin_longitude"]
        },
        {#prende destinazione e scompone in lat long
            lookup=:destination,
            from={
                data=airportsdf_subset,
                key=:iata,
                fields=["latitude", "longitude"]
            },
            as=["dest_latitude", "dest_longitude"]
        }
    ],
    projection={type=:albersUsa},#albersusa è un tipo di rappr mappa degli stati uniti dove l'alaska è rimpicciolito e spostato
                    # e hawaii sono spostate
    longitude="origin_longitude:q",
    latitude="origin_latitude:q",
    longitude2="dest_longitude:q",
    latitude2="dest_latitude:q"
)

In [ ]:
# problema cammino minimo
#con dijkstra
ATL_paths = dijkstra(A,maxdegreeid)
#atl paths è tupla (array[i]= ::float lunghezza cammino per arrivare a i,array[i]= ::int (indice) predecessore di i)

In [ ]:
# prendiamo nodo più distante da atlanta
maximum(ATL_paths[1])

In [ ]:
@show stop1 = argmax(ATL_paths[1])#prendiamo indice nodo con distanza massima
@show uairports[stop1]#chi è?
;

In [ ]:
@show stop2 = ATL_paths[2][stop1]#troviamo indice del predecessore del nodo più distante
@show uairports[stop2]# chi è?
;

In [ ]:
@show stop3 = ATL_paths[2][stop2]#come sopra
@show uairports[stop3]
;

In [ ]:

@show stop4 = ATL_paths[2][stop3]
@show uairports[stop4]
;#siamo arrivati

In [ ]:
#cammino minimo per arrivare all'aereoporto di gst partendo da atl è quindi atl->sea->jnu->gst
#plottiamolo
using VegaLite, VegaDatasets

us10m = dataset("us-10m")
airports = dataset("airports")

@vlplot(width=800, height=500) +
@vlplot(#plot della cartina come prima
    mark={
        :geoshape,
        fill="#eee",
        stroke=:white
    },
    data={
        values=us10m,
        format={
            type=:topojson,
            feature=:states
        }
    },
    projection={type=:albersUsa},
) +
@vlplot(#plot dei nodi(aereoporti)
    :circle,
    data=airportsdf_subset,
    projection={type=:albersUsa},
    longitude="longitude:q",
    latitude="latitude:q",
    size={value=5},
    color={value=:gray}
) +
@vlplot(
    :line,
    data={
        values=[#come dati prendiamo solo gli aereoporti da collegare, in ordine
            {airport=:ATL,order=1},
            {airport=:SEA,order=2},
            {airport=:JNU,order=3},
            {airport=:GST,order=4}
        ]
    },
    transform=[{#trasforma prendendo lat long
        lookup=:airport,
        from={
            data=airports,
            key=:iata,
            fields=["latitude","longitude"]
        }
    }],
    projection={type=:albersUsa},
    longitude="longitude:q",
    latitude="latitude:q",
    order={field=:order,type=:ordinal}#ordine
)

In [ ]:
#possiam metter tutto in una funzione

function find_path(from, to)
    d = dijkstra(A,from)
    shortestpath = zeros(Int,1+Int.(d[1][to]))
    shortestpath[1] = to
    for i = 2:length(shortestpath)
        shortestpath[i] = d[2][shortestpath[i-1]]
    end
    return shortestpath
end

function idToName(id)
    uairports[id]
end

function graph_path(from,to)
    path = idToName.(find_path(from,to))
    orders = 1:length(path)
    start= idToName(from)
    finish = idToName(to)
    mydf = DataFrame(airport=path,order=orders)    
    println("plotto cammino minimo da $(idToName(from)) a $(idToName(to)) ")
    println("itinerario: $(reverse(path))")
    us10m = dataset("us-10m")
    airports = dataset("airports")
    @vlplot(width=800, height=500) +
    @vlplot(#plot della cartina come prima
    mark={
        :geoshape,
        fill="#eee",
        stroke=:white
    },
    data={
        values=us10m,
        format={
            type=:topojson,
            feature=:states
        }
    },
    projection={type=:albersUsa},
) +
@vlplot(#plot dei nodi(aereoporti)
    :circle,
    data=airportsdf_subset,
    projection={type=:albersUsa},
    longitude="longitude:q",
    latitude="latitude:q",
    size={value=5},
    color={value=:gray}
) +
    
    @vlplot(
        :line,
        data=mydf,
              
        
        transform=[{#trasforma prendendo lat long
            
            lookup=:airport,
            from={
                data=airports,
                key=:iata,
                fields=["latitude","longitude"]
            }
        }],
        projection={type=:albersUsa},
        longitude="longitude:q",
        latitude="latitude:q",
        order={field=:order,type=:ordinal}#ordine
    )
end

In [ ]:
graph_path(12,35)

In [ ]:
##### ALBERO DI COPERTURA MINIMO
ti,tj,tv,nverts = mst_prim(A)#creo albero di cop minimo
df_edges = DataFrame(:ei=>uairports[ti],:ej=>uairports[tj])#converto spigoli in dfp

#plotto
@vlplot(width=800, height=500) +
@vlplot(
    mark={
        :geoshape,
        fill="#eee",
        stroke=:white
    },
    data={
        values=us10m,
        format={
            type=:topojson,
            feature=:states
        }
    },
    projection={type=:albersUsa},
) +
@vlplot(
    :circle,
    data=airportsdf_subset,
    projection={type=:albersUsa},
    longitude="longitude:q",
    latitude="latitude:q",
    size={value=20},
    color={value=:gray}
) +
@vlplot(
    :rule,
    data=df_edges, #data=flightsairport,
    transform=[
        {
            lookup=:ei,
            from={
                data=airportsdf_subset,
                key=:iata,
                fields=["latitude", "longitude"]
            },
            as=["originx", "originy"]
        },
        {
            lookup=:ej,
            from={
                data=airportsdf_subset,
                key=:iata,
                fields=["latitude", "longitude"]
            },
            as=["destx", "desty"]
        }
    ],
    projection={type=:albersUsa},
    longitude="originy:q",
    latitude="originx:q",
    longitude2="desty:q",
    latitude2="destx:q"
)

In [ ]:
# è il grafo(albero) con il minimo nuymero di colllegamenti affinchè il grafo sia anocra completamente connesso

# pagerank è un algoritmo che da graduazione a ogni nodo di una rete, usiamolo sulla nostra matrice d adiacenza
v = MatrixNetworks.pagerank(A,0.85)
insertcols!(airportsdf_subset,7,:pagerank_value=>v);#inseriamo i voti nel nostro df di aereoporti

#plottiamo i grtadi rappresentandoli con palline più grandi per gradi più alti
@vlplot(width=500, height=300) +
@vlplot(
    mark={
        :geoshape,
        fill="#eee",
        stroke=:white
    },
    data={
        values=us10m,
        format={
            type=:topojson,
            feature=:states
        }
    },
    projection={type=:albersUsa},
) +
@vlplot(
    :circle,
    data=airportsdf_subset,
    projection={type=:albersUsa},
    longitude="longitude:q",
    latitude="latitude:q",
    size="pagerank_value:q",#attributo che permette di esprimere pagerank come grandezza della pallina
    color={value=:steelblue}
)

In [ ]:
# coeff clustering di un vertice/nodo è quanto vicini sono i suoi vicini ad essere una cricca
cc = clustercoeffs(A)
cc[findall(cc.<=eps())] .= 0
#come prima, aggiungiamo questa graduazione al df e plottiamo
insertcols!(airportsdf_subset,7,:ccvalues=>cc)
@vlplot(width=500, height=300) +
@vlplot(
    mark={
        :geoshape,
        fill="#eee",
        stroke=:white
    },
    data={
        values=us10m,
        format={
            type=:topojson,
            feature=:states
        }
    },
    projection={type=:albersUsa},
) +
@vlplot(
    :circle,
    data=airportsdf_subset,
    projection={type=:albersUsa},
    longitude="longitude:q",
    latitude="latitude:q",
    size="ccvalues:q",
    color={value=:gray}
)